# Testing different network topologies in NEST

This notebook simulates a network of 8 leaky integrate-and-fire (LIF) neurons with different topologies using the NEST simulator.
It includes a function to connect neurons based on specified topologies, such as lattice, small-world, and random.

In [ ]:
# Import the necessary libraries
import nest
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Simulation Setup ---
# Reset the NEST kernel to ensure a clean simulation environment
nest.ResetKernel()

## 2. Network Parameters
Here we define the size of our network and its grid dimensions.

In [ ]:
# Set the simulation time in milliseconds
simulation_time = 100.0  # Adjust this value as needed

# Total number of neurons in the network
num_neurons = 8
# Define the grid dimensions for the square lattice (2 rows, 4 columns)
grid_rows = 2
grid_cols = 4

# Select network topology
# Options are: 'all_to_all', 'ring', 'star', 'random', 'lattice', 'ring_bridge', 'small_world'
selected_topology = 'small_world'

In [ ]:
# Neuron parameters
neuron_params = {
	"E_L": -60.0,  # Resting potential
    "C_m": 250.0,  # Membrane capacitance
    "tau_m": 10.0,  # Membrane time constant
	"t_ref": 2.0,  # Refractory period
    "V_th": -55.0,  # Spike threshold
	"V_reset": -60.0,  # Reset potential after a spike
	"I_e": 0.0,  # No constant input current
}

In [ ]:
# Synapse parameters
input_weight = 1000.0  # Weight for the synaptic connections
synapse_weight = 1000.0  # Weight for the synaptic connections

## 3. Create Network Nodes
We will create the neurons, a spike generator for stimulus, and devices to record activity.

In [ ]:
# Create the 8 neurons using the leaky integrate-and-fire model
# with exponential post-synaptic currents.
neurons = nest.Create("iaf_psc_exp", num_neurons, params=neuron_params)
neurons.V_m = neuron_params["E_L"]  # Initialize membrane potential to resting potential

# Create a spike generator to provide input stimulus to the network
spike_generator = nest.Create("spike_generator")
# Set the spike times for the generator. It will send a single spike at 10.0 ms.
spike_generator.spike_times = [10.0]

# Create a voltmeter to record the membrane potential of the neurons
voltmeter = nest.Create("voltmeter")

# Create a spike recorder to capture spike events
spike_recorder = nest.Create("spike_recorder")

## 4. Connect the Network

In [ ]:
# Function to connect neurons with different topologies
def connect_network_topology(neurons, topology, synapse_weight=1000.0, grid_rows=None, grid_cols=None, 
							p_random=0.3, group_sizes=None, p_shortcut=0.6):
	num_neurons = len(neurons)
	if topology == 'lattice':
		assert grid_rows is not None and grid_cols is not None, "grid_rows and grid_cols required for lattice"
		for i in range(grid_rows):
			for j in range(grid_cols):
				source_idx = i * grid_cols + j
				# Right neighbor
				if j < grid_cols - 1:
					target_idx = source_idx + 1
					nest.Connect(neurons[source_idx], neurons[target_idx], syn_spec={"weight": synapse_weight})
					print(f"Connecting neuron {source_idx} to right neighbor {target_idx}")
				# Left neighbor
				if j > 0:
					target_idx = source_idx - 1
					nest.Connect(neurons[source_idx], neurons[target_idx], syn_spec={"weight": synapse_weight})
					print(f"Connecting neuron {source_idx} to left neighbor {target_idx}")
				# Below neighbor
				if i < grid_rows - 1:
					target_idx = source_idx + grid_cols
					nest.Connect(neurons[source_idx], neurons[target_idx], syn_spec={"weight": synapse_weight})
					print(f"Connecting neuron {source_idx} to below neighbor {target_idx}")
				# Above neighbor
				if i > 0:
					target_idx = source_idx - grid_cols
					nest.Connect(neurons[source_idx], neurons[target_idx], syn_spec={"weight": synapse_weight})
					print(f"Connecting neuron {source_idx} to above neighbor {target_idx}")
	elif topology == 'all_to_all':
		nest.Connect(neurons, neurons, syn_spec={"weight": synapse_weight})
	elif topology == 'ring':
		for i in range(num_neurons):
			nest.Connect(neurons[i], neurons[(i+1)%num_neurons], syn_spec={"weight": synapse_weight})
		nest.Connect(neurons[-1], neurons[0], syn_spec={"weight": synapse_weight})
	elif topology == 'star':
		center = 0
		for i in range(1, num_neurons):
			nest.Connect(neurons[center], neurons[i], syn_spec={"weight": synapse_weight})
			nest.Connect(neurons[i], neurons[center], syn_spec={"weight": synapse_weight})
	elif topology == 'random':
		import random
		for i in range(num_neurons):
			for j in range(num_neurons):
				if i != j and random.random() < p_random:
					nest.Connect(neurons[i], neurons[j], syn_spec={"weight": synapse_weight})
	elif topology == 'bridge':
		if group_sizes is None:
			group_sizes = [num_neurons // 2, num_neurons // 2]
		size_group1, size_group2 = group_sizes
		if size_group1 + size_group2 > len(neurons):
			raise ValueError("Not enough neurons for the specified group sizes.")
		group1 = neurons[:size_group1]
		group2 = neurons[size_group1:size_group1+size_group2]
		# Connect within group1 (all-to-all)
		nest.Connect(group1, group1, syn_spec={"weight": synapse_weight})
		# Connect within group2 (all-to-all)
		nest.Connect(group2, group2, syn_spec={"weight": synapse_weight})
		# Bridge connection: last neuron of group1 to first neuron of group2
		nest.Connect(group1[-1], group2[0], syn_spec={"weight": synapse_weight})
	elif topology == 'ring_bridge':
		if group_sizes is None:
			group_sizes = [num_neurons // 2, num_neurons // 2]
		size_group1, size_group2 = group_sizes
		if size_group1 + size_group2 > len(neurons):
			raise ValueError("Not enough neurons for the specified group sizes.")
		group1 = neurons[:size_group1]
		group2 = neurons[size_group1:size_group1+size_group2]
		# Connect within group1 (ring)
		for i in range(size_group1):
			nest.Connect(group1[i], group1[(i+1)%size_group1], syn_spec={"weight": synapse_weight})
		nest.Connect(group1[-1], group1[0], syn_spec={"weight": synapse_weight})
		# Connect within group2 (ring)
		for i in range(size_group2):
			nest.Connect(group2[i], group2[(i+1)%size_group2], syn_spec={"weight": synapse_weight})
		nest.Connect(group2[-1], group2[0], syn_spec={"weight": synapse_weight})
		# Bridge connection: last neuron of group1 to first neuron of group2
		nest.Connect(group1[-1], group2[0], syn_spec={"weight": synapse_weight})
	elif topology == 'small_world':
		# Connect as a ring first
		for i in range(num_neurons):
			nest.Connect(neurons[i], neurons[(i+1)%num_neurons], syn_spec={"weight": synapse_weight})
		nest.Connect(neurons[-1], neurons[0], syn_spec={"weight": synapse_weight})
		# Connect for small-world topology, add shortcuts
		for i in range(num_neurons):
			for j in range(num_neurons):
				target_idx = (i + j) % num_neurons
				if np.random.rand() < p_shortcut:
					nest.Connect(neurons[i], neurons[target_idx], syn_spec={"weight": synapse_weight})
					print(f"Connecting neuron {i} to shortcut {target_idx}")
	else:
		raise ValueError(f"Unknown topology: {topology}")

In [ ]:
# Connect neurons using the selected topology
connect_network_topology(
    neurons,
    selected_topology,
    synapse_weight=synapse_weight,
    grid_rows=grid_rows,
    grid_cols=grid_cols,
    p_random=0.3
)
print(f"Connections established using topology: {selected_topology}")

## 5. Connect Devices
Now we connect the spike generator and recording devices to the appropriate neurons.

In [ ]:
# Connect the spike generator to the first neuron in the network (index 0).
# This will be the entry point for the stimulus.
nest.Connect(spike_generator, neurons[0], syn_spec={"weight": input_weight})

# Connect the voltmeter to all neurons to record their membrane potential.
nest.Connect(voltmeter, neurons)

# Connect all neurons to the spike recorder.
nest.Connect(neurons, spike_recorder)

## 6. Run the Simulation

In [ ]:
# Simulate the network
nest.Simulate(simulation_time)

## 7. Visualize the Results
Finally, we plot the membrane potentials and spike times recorded by our devices to observe the network's activity.

In [ ]:
# Side-by-side raster plots: original and sorted by first spike time, with neuron IDs
events = nest.GetStatus(spike_recorder, "events")[0]
senders = np.array(events["senders"])
times = np.array(events["times"])

# --- Prepare sorted neuron order and labels ---
first_spike_times = []
for n in range(1, num_neurons + 1):
    neuron_times = times[senders == n]
    if len(neuron_times) > 0:
        first_spike_times.append((n, neuron_times[0]))
    else:
        first_spike_times.append((n, np.nan))
first_spike_times_sorted = sorted(first_spike_times, key=lambda x: (np.nan_to_num(x[1], nan=1e9)))
sorted_ids = [x[0] for x in first_spike_times_sorted]
sorted_labels = [f"Neuron {nid}" for i, nid in enumerate(sorted_ids)]

fig, axes = plt.subplots(1, 2, figsize=(18, 7), sharex=True)

# --- Original raster ---
axes[0].set_title('Spike Raster Plot (Neuron ID order)')
line_len = 0.8
axes[0].vlines(times, senders - line_len / 2, senders + line_len / 2, color='k', linewidth=0.8)
axes[0].axvline(10.0, color='r', linestyle='--', label='Stimulus')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Neuron ID')
axes[0].set_yticks(range(1, num_neurons + 1))
axes[0].set_ylim(0.5, num_neurons + 0.5)
axes[0].legend()

# --- Sorted raster ---
for idx, n in enumerate(sorted_ids):
    neuron_times = times[senders == n]
    axes[1].vlines(neuron_times, idx + 1 - 0.4, idx + 1 + 0.4, color='k', linewidth=0.8)
axes[1].axvline(10.0, color='r', linestyle='--', label='Stimulus')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Neuron (sorted by activation)')
axes[1].set_title('Spike Raster Plot (Sorted by First Spike Time)')
axes[1].set_yticks(range(1, num_neurons + 1))
axes[1].set_yticklabels(sorted_labels)
axes[1].set_ylim(0.5, num_neurons + 0.5)
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Animation with circles for neurons, flashing on spike
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Get spike times for each neuron
events = nest.GetStatus(spike_recorder, "events")[0]
senders = np.array(events["senders"])
times = np.array(events["times"])
spike_times_grid = [[[] for _ in range(grid_cols)] for _ in range(grid_rows)]
for n in range(1, num_neurons + 1):
    neuron_times = times[senders == n]
    row = (n - 1) // grid_cols
    col = (n - 1) % grid_cols
    spike_times_grid[row][col] = neuron_times

# Animation parameters
t_min = 0
t_max = simulation_time
dt = 0.5
frames = int((t_max - t_min) / dt) + 1
time_points = np.linspace(t_min, t_max, frames)

fig, ax = plt.subplots(figsize=(8, 4))
ax.set_xlim(-0.5, grid_cols-0.5)
ax.set_ylim(-0.5, grid_rows-0.5)
ax.set_xticks(range(grid_cols))
ax.set_yticks(range(grid_rows))
ax.set_xlabel('Column')
ax.set_ylabel('Row')
ax.set_title('Spike Propagation Animation (Circles)')

# Draw circles for neurons
circles = []
for i in range(grid_rows):
    for j in range(grid_cols):
        circle = plt.Circle((j, i), 0.3, color='gray', ec='black')
        ax.add_patch(circle)
        circles.append(circle)

def update(frame):
    t = time_points[frame]
    for idx, circle in enumerate(circles):
        i = idx // grid_cols
        j = idx % grid_cols
        # Flash if neuron spikes within this frame
        if any((st > t-dt/2) and (st <= t+dt/2) for st in spike_times_grid[i][j]):
            circle.set_color('yellow')
            circle.set_radius(0.4)
        else:
            circle.set_color('gray')
            circle.set_radius(0.3)
    ax.set_title(f'Spike Propagation at t={t:.1f} ms')
    return circles

ani = FuncAnimation(fig, update, frames=frames, interval=100, blit=True)
plt.close(fig)

HTML(ani.to_jshtml())